In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/My\ Drive/giraffe/scenenet.zip scenenet.zip
!unzip scenenet.zip > ziplog.txt


In [0]:
import torch
from torch import nn
from torch.utils.data import random_split, Dataset, DataLoader
from torchvision import transforms
from torch.autograd import Variable
import torchvision
from torchvision.datasets import MNIST, ImageFolder
import numpy as np
import h5py
from PIL import Image

In [0]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 3, 6, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [0]:
class RayTracingDataset(Dataset):

    def __init__(self, renders, transform=None):
        '''
        Args:
            renders (Numpy array)
            transform (callable, optional)
        '''
        self.renders = renders
        self.transform = transform
    
    def __len__(self):
        return len(self.renders)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample = {'render': self.renders[idx]}

        return sample


In [0]:
import os

transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [50]:
b_size = 128
img_size = 128

train_dataset = ImageFolder('root', transform=transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=b_size, shuffle=True, num_workers=4,
)

print(len(train_loader))


num_epochs = 500
learning_rate = 1e-3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = autoencoder()
model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, 
    weight_decay=1e-5
)


8


In [0]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 3, img_size, img_size)
    return x

from torchvision.utils import save_image

losses = []
for epoch in range(num_epochs):
    for data in train_loader:

        data = data[0]
        data = data.view(data.size(0), 3, img_size, img_size)
        data = Variable(data).to(device)

        optimizer.zero_grad()

        output = model(data)

        loss = criterion(output, data)
        losses.append(loss)

        loss.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.data))
        pic = to_img(output.cpu().data)
        save_image(pic, '/content/drive/My Drive/giraffe/train/{}.png'.format(str(epoch+1)))
import matplotlib.pyplot as plt

plt.plot(losses)
plt.show()
torch.save(model.state_dict(), '/content/drive/My Drive/giraffe/train/checkpoint.pth')

epoch [1/500], loss:0.0666
epoch [11/500], loss:0.0610
epoch [21/500], loss:0.0594
epoch [31/500], loss:0.0621
epoch [41/500], loss:0.0503
epoch [51/500], loss:0.0443
epoch [61/500], loss:0.0498
epoch [71/500], loss:0.0438
epoch [81/500], loss:0.0434
epoch [91/500], loss:0.0461
epoch [101/500], loss:0.0483
epoch [111/500], loss:0.0376
epoch [121/500], loss:0.0391
epoch [131/500], loss:0.0339
epoch [141/500], loss:0.0333
epoch [151/500], loss:0.0283
epoch [161/500], loss:0.0328
epoch [171/500], loss:0.0284
epoch [181/500], loss:0.0274
epoch [191/500], loss:0.0249
epoch [201/500], loss:0.0251
epoch [211/500], loss:0.0266
epoch [221/500], loss:0.0253
epoch [231/500], loss:0.0278
epoch [241/500], loss:0.0258
epoch [251/500], loss:0.0282
epoch [261/500], loss:0.0249
epoch [271/500], loss:0.0224
epoch [281/500], loss:0.0235
epoch [291/500], loss:0.0250
epoch [301/500], loss:0.0270
epoch [311/500], loss:0.0255
epoch [321/500], loss:0.0277
epoch [331/500], loss:0.0239
epoch [341/500], loss:0.0